### Objective:
- To explore interpolation error at different values of radius.
- Select the radius for interpolation purpose keeping a balance between interpolation error and no gaps are created in flow fields across the geometry.
-  Patches indicate incomplete or missing interpolated data. This can happen if the radius used for interpolation is too small to cover all necessary points or cells in certain regions. When the radius is too small, not enough neighboring points are included in the interpolation process, leading to areas with incomplete coverage or "gaps."

### Import all necessary libraries

In [11]:
import pyvista as pv
import os
import pandas as pd
from sklearn.metrics import mean_squared_error,root_mean_squared_error,r2_score,mean_absolute_error,accuracy_score
import numpy as np

### Define the function for reading any mesh data

In [12]:
#Check if foam file exists or not
def _get_foam_file(file_path):
    for file in os.listdir(file_path):
        if file.endswith('.foam'):
            return os.path.join(file_path, file)

#Read mesh data using pyvista open foam reader
def _read_mesh_data(work_dir):

    reader = pv.OpenFOAMReader(_get_foam_file(work_dir))
    last_time_step = int(reader.time_values[-1])
    reader.set_active_time_value(last_time_step)
    # reader.cell_to_point_creation = False

    internal_mesh = reader.read()["internalMesh"]
    boundaries = reader.read()["boundary"]

    internal_mesh.work_dir = work_dir
    internal_mesh.active_time_value = last_time_step

    return internal_mesh, boundaries

#Store the selected flow fields in the mesh
def _select_only_relevant_field_data(pvObject, fields):

    pvCopy = pvObject.copy()
    pvCopy.clear_data()
    for field in fields:
        if field in pvObject.array_names:
            pvCopy[field] = pvObject[field]
    
    pvCopy = pvCopy.cell_data_to_point_data() #converting cell data to point data
    return pvCopy

fields = ['epsilon', 'U', 'mag(U)','k', 'strainRate']

### Read the largest geometry mesh

In [13]:
largest_mesh_dir=r"D:\GPOD ANN Sartorius BioStat 50L 30 degrees removed geoms modified v3\Sartorius BioStat 50L\BIOSTAT_50L_Run_41"
largest_mesh,_=_read_mesh_data(largest_mesh_dir)
largest_mesh.clear_data()
largest_mesh

UnstructuredGrid,Information
N Cells,1004477
N Points,1185213
X Bounds,"-1.850e-01, 1.850e-01"
Y Bounds,"-1.850e-01, 1.850e-01"
Z Bounds,"-9.250e-02, 4.039e-01"
N Arrays,0


### Read the target mesh that is to be interpolated

In [14]:
target_mesh_dir=r"D:\GPOD ANN Sartorius BioStat 50L 30 degrees removed geoms modified v3\Sartorius BioStat 50L\BIOSTAT_50L_Run_07"
internal_mesh,_=_read_mesh_data(target_mesh_dir)
target_mesh=_select_only_relevant_field_data(internal_mesh, fields)
target_mesh

UnstructuredGrid (0x1a187b342e0)
  N Cells:	832347
  N Points:	977284
  X Bounds:	-1.850e-01, 1.850e-01
  Y Bounds:	-1.850e-01, 1.850e-01
  Z Bounds:	-9.250e-02, 2.359e-01
  N Arrays:	5

Note: The X bounds, Y bounds and Z bounds are for the entire UnstructuredGrid object. They define the overall spatial extent of the whole mesh, encompassing all cells and points within it.

### Interpolate the target mesh against largest geometry with radius=0.001

In [15]:
interp_mesh = largest_mesh.interpolate(target_mesh, radius = 1e-3, progress_bar = True)

Interpolating: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████[00:01<00:00]


In [16]:
interp_mesh = interp_mesh.point_data_to_cell_data()
interp_mesh

UnstructuredGrid (0x1a187b34760)
  N Cells:	1004477
  N Points:	1185213
  X Bounds:	-1.850e-01, 1.850e-01
  Y Bounds:	-1.850e-01, 1.850e-01
  Z Bounds:	-9.250e-02, 4.039e-01
  N Arrays:	5

In [27]:
#Visualizing epsilon,mag(U),k,strainRate flow fields

pl = pv.Plotter(shape=(2,2),notebook=False)

pl.set_background('white')
pl.add_mesh(interp_mesh.slice(normal='y'),scalars="mag(U)",clim=[1e-3,1],scalar_bar_args={'color':'black'},log_scale=True)
pl.view_xz()

pl.subplot(0,1)
pl.add_mesh(interp_mesh.slice(normal='y'),scalars="k",clim=[1e-3,1],scalar_bar_args={'color':'black'},log_scale=True)
pl.view_xz()

pl.subplot(1,0)
pl.add_mesh(interp_mesh.slice(normal='y'),scalars="strainRate",clim=[0.5,100],scalar_bar_args={'color':'black'},log_scale=True)
pl.view_xz()

pl.subplot(1,1)
pl.add_mesh(interp_mesh.slice(normal='y'),scalars="epsilon",clim=[1e-3,1],scalar_bar_args={'color':'black'},log_scale=True)
pl.view_xz()

pl.show()

Note: Some patches can be seen in between of the flow fields.

### Interpolate the target mesh against largest geometry with radius=0.0155

Note: The minimum of the three calculated dimensions (X, Y, and Z) is chosen as the radius. This is probably done to ensure that the interpolation uses the smallest spatial extent, which might be relevant for ensuring uniform interpolation across the entire mesh.Finally, the radius is multiplied by 5, likely to scale the interpolation area to cover a broader region around the mesh for interpolation purposes.

In [29]:
bounds = target_mesh.extract_cells(1).bounds #extracting the mesh bounds
print([bounds[i+1]-bounds[i] for i in [0,2,4]])
radius = min([bounds[i+1]-bounds[i] for i in [0,2,4]]) 
print(radius*5)#defining the radius for interpolation

[0.0035127103328704834, 0.003493204712867737, 0.0031145475804805756]
0.015572737902402878


In [30]:
interp_mesh_v2 = largest_mesh.interpolate(target_mesh, radius = radius*5, progress_bar = True)
interp_mesh_v2 = interp_mesh_v2.point_data_to_cell_data()

Interpolating: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████[00:59<00:00]


In [31]:
interp_mesh_v2

UnstructuredGrid (0x1a1ccd84b20)
  N Cells:	1004477
  N Points:	1185213
  X Bounds:	-1.850e-01, 1.850e-01
  Y Bounds:	-1.850e-01, 1.850e-01
  Z Bounds:	-9.250e-02, 4.039e-01
  N Arrays:	5

In [32]:
#Visualizing epsilon,mag(U),k,strainRate flow fields

pl = pv.Plotter(shape=(2,2),notebook=False)

pl.set_background('white')
pl.add_mesh(interp_mesh_v2.slice(normal='y'),scalars="mag(U)",clim=[1e-3,1],scalar_bar_args={'color':'black'},log_scale=True)
pl.view_xz()

pl.subplot(0,1)
pl.add_mesh(interp_mesh_v2.slice(normal='y'),scalars="k",clim=[1e-3,1],scalar_bar_args={'color':'black'},log_scale=True)
pl.view_xz()

pl.subplot(1,0)
pl.add_mesh(interp_mesh_v2.slice(normal='y'),scalars="strainRate",clim=[0.5,100],scalar_bar_args={'color':'black'},log_scale=True)
pl.view_xz()

pl.subplot(1,1)
pl.add_mesh(interp_mesh_v2.slice(normal='y'),scalars="epsilon",clim=[1e-3,1],scalar_bar_args={'color':'black'},log_scale=True)
pl.view_xz()

pl.show()

Note: No patch as such can be seen in between the flow fields

### Interpolate the target mesh against largest geometry with radius=0.0311

In [33]:
print(radius*10)

0.031145475804805756


In [34]:
interp_mesh_v3 = largest_mesh.interpolate(target_mesh, radius = radius*10, progress_bar = True)
interp_mesh_v3 = interp_mesh_v3.point_data_to_cell_data()

Interpolating: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████[09:11<00:00]


In [35]:
interp_mesh_v3

UnstructuredGrid (0x1a1ccdd7c40)
  N Cells:	1004477
  N Points:	1185213
  X Bounds:	-1.850e-01, 1.850e-01
  Y Bounds:	-1.850e-01, 1.850e-01
  Z Bounds:	-9.250e-02, 4.039e-01
  N Arrays:	5

In [36]:
#Visualizing epsilon,mag(U),k,strainRate flow fields

pl = pv.Plotter(shape=(2,2),notebook=False)

pl.set_background('white')
pl.add_mesh(interp_mesh_v3.slice(normal='y'),scalars="mag(U)",clim=[1e-3,1],scalar_bar_args={'color':'black'},log_scale=True)
pl.view_xz()

pl.subplot(0,1)
pl.add_mesh(interp_mesh_v3.slice(normal='y'),scalars="k",clim=[1e-3,1],scalar_bar_args={'color':'black'},log_scale=True)
pl.view_xz()

pl.subplot(1,0)
pl.add_mesh(interp_mesh_v3.slice(normal='y'),scalars="strainRate",clim=[0.5,100],scalar_bar_args={'color':'black'},log_scale=True)
pl.view_xz()

pl.subplot(1,1)
pl.add_mesh(interp_mesh_v3.slice(normal='y'),scalars="epsilon",clim=[1e-3,1],scalar_bar_args={'color':'black'},log_scale=True)
pl.view_xz()

pl.show()

Note: No patch as such can be seen in between the flow fields

### Interpolate the target mesh against largest geometry with radius=0.05

In [37]:
interp_mesh_v4 = largest_mesh.interpolate(target_mesh, radius = 0.05, progress_bar = True)
interp_mesh_v4 = interp_mesh_v4.point_data_to_cell_data()

Interpolating: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████[24:23<00:00]


In [38]:
interp_mesh_v4

UnstructuredGrid (0x1a1cce04160)
  N Cells:	1004477
  N Points:	1185213
  X Bounds:	-1.850e-01, 1.850e-01
  Y Bounds:	-1.850e-01, 1.850e-01
  Z Bounds:	-9.250e-02, 4.039e-01
  N Arrays:	5

In [39]:
#Visualizing epsilon,mag(U),k,strainRate flow fields

pl = pv.Plotter(shape=(2,2),notebook=False)

pl.set_background('white')
pl.add_mesh(interp_mesh_v4.slice(normal='y'),scalars="mag(U)",clim=[1e-3,1],scalar_bar_args={'color':'black'},log_scale=True)
pl.view_xz()

pl.subplot(0,1)
pl.add_mesh(interp_mesh_v4.slice(normal='y'),scalars="k",clim=[1e-3,1],scalar_bar_args={'color':'black'},log_scale=True)
pl.view_xz()

pl.subplot(1,0)
pl.add_mesh(interp_mesh_v4.slice(normal='y'),scalars="strainRate",clim=[0.5,100],scalar_bar_args={'color':'black'},log_scale=True)
pl.view_xz()

pl.subplot(1,1)
pl.add_mesh(interp_mesh_v4.slice(normal='y'),scalars="epsilon",clim=[1e-3,1],scalar_bar_args={'color':'black'},log_scale=True)
pl.view_xz()

pl.show()

Note: A little bit patches can be seen in between the flow fields.

### Interpolate the target mesh against largest geometry with radius=0.008

In [40]:
interp_mesh_v5 = largest_mesh.interpolate(target_mesh, radius = 0.008, progress_bar = True)
interp_mesh_v5

Interpolating: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████[00:16<00:00]


UnstructuredGrid (0x1a1ccde39a0)
  N Cells:	1004477
  N Points:	1185213
  X Bounds:	-1.850e-01, 1.850e-01
  Y Bounds:	-1.850e-01, 1.850e-01
  Z Bounds:	-9.250e-02, 4.039e-01
  N Arrays:	5

In [41]:
interp_mesh_v5 = interp_mesh_v5.point_data_to_cell_data()

In [42]:
#Visualizing epsilon,mag(U),k,strainRate flow fields

pl = pv.Plotter(shape=(2,2),notebook=False)

pl.set_background('white')
pl.add_mesh(interp_mesh_v5.slice(normal='y'),scalars="mag(U)",clim=[1e-3,1],scalar_bar_args={'color':'black'},log_scale=True)
pl.view_xz()

pl.subplot(0,1)
pl.add_mesh(interp_mesh_v5.slice(normal='y'),scalars="k",clim=[1e-3,1],scalar_bar_args={'color':'black'},log_scale=True)
pl.view_xz()

pl.subplot(1,0)
pl.add_mesh(interp_mesh_v5.slice(normal='y'),scalars="strainRate",clim=[0.5,100],scalar_bar_args={'color':'black'},log_scale=True)
pl.view_xz()

pl.subplot(1,1)
pl.add_mesh(interp_mesh_v5.slice(normal='y'),scalars="epsilon",clim=[1e-3,1],scalar_bar_args={'color':'black'},log_scale=True)
pl.view_xz()

pl.show()

Note: No patches can be seen in between the flow fields

### Interpolate the target mesh against largest geometry with radius=0.0075

In [43]:
interp_mesh_v6 = largest_mesh.interpolate(target_mesh, radius = 0.0075, progress_bar = True)
interp_mesh_v6=interp_mesh_v6.point_data_to_cell_data()
interp_mesh_v6

Interpolating: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████[00:13<00:00]


UnstructuredGrid (0x1a1d6ddf940)
  N Cells:	1004477
  N Points:	1185213
  X Bounds:	-1.850e-01, 1.850e-01
  Y Bounds:	-1.850e-01, 1.850e-01
  Z Bounds:	-9.250e-02, 4.039e-01
  N Arrays:	5

In [44]:
#Visualizing epsilon,mag(U),k,strainRate flow fields

pl = pv.Plotter(shape=(2,2),notebook=False)

pl.set_background('white')
pl.add_mesh(interp_mesh_v6.slice(normal='y'),scalars="mag(U)",clim=[1e-3,1],scalar_bar_args={'color':'black'},log_scale=True)
pl.view_xz()

pl.subplot(0,1)
pl.add_mesh(interp_mesh_v6.slice(normal='y'),scalars="k",clim=[1e-3,1],scalar_bar_args={'color':'black'},log_scale=True)
pl.view_xz()

pl.subplot(1,0)
pl.add_mesh(interp_mesh_v6.slice(normal='y'),scalars="strainRate",clim=[0.5,100],scalar_bar_args={'color':'black'},log_scale=True)
pl.view_xz()

pl.subplot(1,1)
pl.add_mesh(interp_mesh_v6.slice(normal='y'),scalars="epsilon",clim=[1e-3,1],scalar_bar_args={'color':'black'},log_scale=True)
pl.view_xz()

pl.show()

Note: No patches can be seen in between the flow fields.

### Interpolate the target mesh against largest geometry with radius=0.005

In [46]:
interp_mesh_v7 = largest_mesh.interpolate(target_mesh, radius = 0.005, progress_bar = True)
interp_mesh_v7=interp_mesh_v7.point_data_to_cell_data()
interp_mesh_v7

Interpolating: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████[00:03<00:00]


UnstructuredGrid (0x1a1ccd84d00)
  N Cells:	1004477
  N Points:	1185213
  X Bounds:	-1.850e-01, 1.850e-01
  Y Bounds:	-1.850e-01, 1.850e-01
  Z Bounds:	-9.250e-02, 4.039e-01
  N Arrays:	5

In [68]:
#Visualizing epsilon,mag(U),k,strainRate flow fields

pl = pv.Plotter(shape=(2,2),notebook=False)

pl.set_background('white')
pl.add_mesh(interp_mesh_v7.slice(normal='y'),scalars="mag(U)",clim=[1e-3,1],scalar_bar_args={'color':'black'},log_scale=True)
pl.view_xz()

pl.subplot(0,1)
pl.add_mesh(interp_mesh_v7.slice(normal='y'),scalars="k",clim=[1e-3,1],scalar_bar_args={'color':'black'},log_scale=True)
pl.view_xz()

pl.subplot(1,0)
pl.add_mesh(interp_mesh_v7.slice(normal='y'),scalars="strainRate",clim=[0.5,100],scalar_bar_args={'color':'black'},log_scale=True)
pl.view_xz()

pl.subplot(1,1)
pl.add_mesh(interp_mesh_v7.slice(normal='y'),scalars="epsilon",clim=[1e-3,1],scalar_bar_args={'color':'black'},log_scale=True)
pl.view_xz()

pl.show()

Note: Patches are visible.

### Code snippet to calculate regression metrics

In [49]:
#Store all the regression metrics in tabular format
def calc_regression_metrics(target_mesh,interp_mesh,true_idx,radius):
    
    df_metrics=pd.DataFrame(index=["R2_score","MSE","RMSE","MAE"])
    
    for field in fields:
        if field == 'U':
            for i in range(3):
                true_values=np.round(target_mesh[f"{field}"][:,i],4)
                pred_values=np.round(interp_mesh[f"{field}"][true_idx][:,i],4)

                r2_score_value=r2_score(true_values,pred_values)
                mse_value=mean_squared_error(true_values,pred_values)
                rmse_value=root_mean_squared_error(true_values,pred_values)
                mae_value=mean_absolute_error(true_values,pred_values)

                df_metrics[f"{field}_{i}"]=[r2_score_value,mse_value,rmse_value,mae_value]

        true_values=np.round(target_mesh[f"{field}"],4)
        #print(true_values.shape)
        pred_values=np.round(interp_mesh[f"{field}"][true_idx],4)
        #print(pred_values.shape)
                
        r2_score_value=r2_score(true_values,pred_values)
        mse_value=mean_squared_error(true_values,pred_values)
        rmse_value=root_mean_squared_error(true_values,pred_values)
        mae_value=mean_absolute_error(true_values,pred_values)
        
        df_metrics[f"{field}"]=[r2_score_value,mse_value,rmse_value,mae_value]
        
        df_metrics.to_excel(f"Regression_mterics_with_radius_{radius}.xlsx")

In [50]:
target_mesh_copy=target_mesh.copy()
target_mesh_copy=target_mesh_copy.point_data_to_cell_data()
target_mesh_copy.cell_centers().points.shape

(832347, 3)

In [51]:
idx=interp_mesh.find_containing_cell(target_mesh_copy.cell_centers().points)
print(idx.shape)
radius=0.001

calc_regression_metrics(target_mesh_copy,interp_mesh,idx,f"{radius}")

(832347,)


In [52]:
idx=interp_mesh_v2.find_containing_cell(target_mesh_copy.cell_centers().points)
print(idx.shape)
radius=0.0155

calc_regression_metrics(target_mesh_copy,interp_mesh_v2,idx,f"{radius}")

(832347,)


In [53]:
idx=interp_mesh_v3.find_containing_cell(target_mesh_copy.cell_centers().points)
print(idx.shape)
radius=0.0311

calc_regression_metrics(target_mesh_copy,interp_mesh_v3,idx,f"{radius}")

(832347,)


In [54]:
idx=interp_mesh_v4.find_containing_cell(target_mesh_copy.cell_centers().points)
print(idx.shape)
radius=0.05

calc_regression_metrics(target_mesh_copy,interp_mesh_v4,idx,f"{radius}")

(832347,)


In [55]:
idx=interp_mesh_v5.find_containing_cell(target_mesh_copy.cell_centers().points)
print(idx.shape)
radius=0.008

calc_regression_metrics(target_mesh_copy,interp_mesh_v5,idx,f"{radius}")

(832347,)


In [60]:
idx=interp_mesh_v6.find_containing_cell(target_mesh_copy.cell_centers().points)
print(idx.shape)
radius=0.0075

calc_regression_metrics(target_mesh_copy,interp_mesh_v6,idx,f"{radius}")

(832347,)


In [61]:
idx=interp_mesh_v7.find_containing_cell(target_mesh_copy.cell_centers().points)
print(idx.shape)
radius=0.005

calc_regression_metrics(target_mesh_copy,interp_mesh_v7,idx,f"{radius}")

(832347,)


### Visualizing how true mesh looks like

In [62]:
# Visualizing how true mesh looks like
pl = pv.Plotter()
pl.set_background('white')
pl.add_mesh(target_mesh_copy.slice(normal='y'),scalars="mag(U)",clim=[1e-3,1],scalar_bar_args={'color':'black'})
pl.view_xz()
pl.show()

ViewInteractiveWidget(height=768, layout=Layout(height='auto', width='100%'), width=1024)

In [63]:
# Superimpose the interpolated points at idx
import time

start=time.time()
idx_v2=interp_mesh_v2.find_containing_cell(target_mesh_copy.cell_centers().points)
true_idx=target_mesh_copy.find_containing_cell(target_mesh_copy.cell_centers().points)

interp_mesh_v2_copy=interp_mesh_v2.copy()
interp_mesh_v2_copy.clear_data()

for field in ['strainRate', 'epsilon', 'k', 'U', 'mag(U)']:
    #Get the flow field data of both meshes
    interp_field=interp_mesh_v2[field]
    true_field=target_mesh_copy[field]
    
    interp_field[idx_v2]=true_field[true_idx]
    
    interp_mesh_v2_copy[field]=interp_field
    
end=time.time()
print(end-start)

23.81863832473755


In [64]:
# Visualizing how true mesh looks like after sumperimposing
pl = pv.Plotter()
pl.set_background('white')
pl.add_mesh(interp_mesh_v2_copy.slice(normal='y'),scalars="mag(U)",clim=[1e-3,1],scalar_bar_args={'color':'black'})
pl.view_xz()
pl.show()

ViewInteractiveWidget(height=768, layout=Layout(height='auto', width='100%'), width=1024)

In [67]:
#Visualizing epsilon,mag(U),k,strainRate flow fields

pl = pv.Plotter(shape=(2,2),notebook=False)

pl.set_background('white')
pl.add_mesh(interp_mesh_v2.slice(normal='y'),scalars="mag(U)",clim=[1e-3,1],scalar_bar_args={'color':'black'},log_scale=True)
pl.view_xz()

pl.subplot(0,1)
pl.add_mesh(interp_mesh_v2.slice(normal='y'),scalars="k",clim=[1e-3,1],scalar_bar_args={'color':'black'},log_scale=True)
pl.view_xz()

pl.subplot(1,0)
pl.add_mesh(interp_mesh_v2.slice(normal='y'),scalars="strainRate",clim=[0.5,100],scalar_bar_args={'color':'black'},log_scale=True)
pl.view_xz()

pl.subplot(1,1)
pl.add_mesh(interp_mesh_v2.slice(normal='y'),scalars="epsilon",clim=[1e-3,1],scalar_bar_args={'color':'black'},log_scale=True)
pl.view_xz()

pl.show()